In [71]:
import string

In [2]:
import nltk
from nltk.corpus import stopwords

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import numpy as np
import pandas as pd

## Working on the newly created data

In [96]:
df = pd.read_csv("C:/Users/hp/Desktop/Test.csv")
df.head()

,SUBJECT,TOPIC,QUESTION
0,operating system,fcfs,NaN
1,NaN,NaN,a) the number on the overview screen correspon...
2,NaN,NaN,? case in which fcfs is the best algo
3,NaN,NaN,to effectively handle security you want to mi...
4,NaN,NaN,38. security functionality is provided by whic...


In [97]:
len(df)

730

Removing non ascii characters and filling NaN with white space

In [98]:
df.fillna(" ",inplace=True)
df["QUESTION"] = df["QUESTION"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
df.head()

,SUBJECT,TOPIC,QUESTION
0,operating system,fcfs,
1,,,a) the number on the overview screen correspon...
2,,,? case in which fcfs is the best algo
3,,,to effectively handle security you want to mi...
4,,,38. security functionality is provided by whic...


Creating a dictionary of topics and empty lists to be filled

In [187]:
d = {}
topic_df = df[df["TOPIC"]!=" "]["TOPIC"]
topic_df.reset_index(drop=True,inplace=True)
x =0
while x<len(topic_df):
    d.update({topic_df[x]:[]})
    x = x+1
x =0

Removing punctuations and numbers; removing redundant words in same sentence and filling the dictionary with questions relating to topic

In [ ]:
exclude = set(string.punctuation)
while x<len(df):
    if df['QUESTION'][x] == " ":
        topic = df['TOPIC'][x]
    else:
        p =df['QUESTION'][x]
        re = string.maketrans(string.punctuation,' '*len(string.punctuation))
        p = p.translate(re)
        r = "".join([o for o in p if not (o.isdigit() or o[0] == '-' and o[1:].isdigit())])
        d[topic].append(" ".join(list(set(r.split(" ")))))
    x = x+1

Creating dataframe of topics

In [239]:
secondary = df[df['QUESTION']== " "]['TOPIC']
secondary.reset_index(drop=True,inplace=True)
secondary=secondary.to_frame()
l = np.arange(len(secondary))
secondary.set_index(l)

,TOPIC
0,fcfs
1,dml
2,jdbc
3,joints
4,buffering
5,applet
6,deadlock
7,ddl
8,polymorphism
9,caching


Creating new column for words to match

In [279]:
x ="MAtCH_WORDS"
secondary[x]=[[] for k in secondary.TOPIC]

In [245]:
#secondary.drop('MAtCH WORDS',axis =1,inplace =True)

In [280]:
secondary

,TOPIC,MAtCH_WORDS
0,fcfs,[]
1,dml,[]
2,jdbc,[]
3,joints,[]
4,buffering,[]
5,applet,[]
6,deadlock,[]
7,ddl,[]
8,polymorphism,[]
9,caching,[]


Using CountVectorizer to check frequency of specific words; training on vocabulary

In [270]:
s =0
while s<len(secondary):
    cv = CountVectorizer(stop_words= 'english')
    cvtrain = cv.fit_transform(d[secondary.TOPIC[s]])
    b =cv.transform(d[secondary.TOPIC[s]])
    
    
    a =cvtrain.toarray()
    
    total = np.zeros(len(cv.get_feature_names()))
    
    #calculating 
    x = 0
    y = 0
    feature = cv.get_feature_names()
    for x in range(len(feature)):
        for y in range(len(a)):
            total[x] = total[x] + cvtrain[y,x]
    
    #arranging the indexes wrt to decreasing order of values obtained
    index = np.zeros(len(total))
    for x in range(len(total)):
        index[x] = np.argmax(total)
        total[np.argmax(total)]= -1
    #storing top 10 most frequently used words wrt a topic  
    for x in index[:10]:
        secondary.MAtCH_WORDS[s].append(feature[int(x)])
    print s
    s = s+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


Reading questions dataset and making appropriate changes for processing

In [262]:
question_bank = pd.read_csv("C:/Users/hp/Desktop/500_questions.csv")
question_bank['question'] = question_bank['question'].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

question_bank.drop('Unnamed: 0',axis = 1, inplace = True)
x = 0
while x<question_bank.shape[0]:
    question_bank.iloc[x].question = question_bank.iloc[x].question.lower()
    x = x+1

In [282]:
question_bank

,question,fcfs,dml,jdbc,joints,buffering,applet,deadlock,ddl,polymorphism,caching,thread,security,scheduling,inheritance,encapsulation,transaction,object,sql,keys
0,(avg. of 42 ) : what is a sql view?,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,1,1
1,question involving primary key integrity const...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2
2,how secure are encrypted stored procedures tr...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,8. a candidate appearing for an examinatio n ...,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0
4,consider the following areas for tuning. the o...,1,3,0,1,4,1,1,3,0,4,0,2,1,1,2,3,1,3,2
5,2. why we cant create object for abstract class?,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,3,0,0
6,6.transfering of pages from memory to peripher...,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0
7,there are 30 questions in which 2 are star que...,0,3,0,2,0,1,0,0,0,0,2,1,0,0,0,0,1,1,3
8,a) infopackage ' scheduler ' repair reques...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,an anchoring object is a print condition objec...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0


Topicwise scoring initialisation to 0

In [266]:
x = 0
while x<len(secondary):
    topic = secondary.TOPIC[x]
    question_bank[topic]=0
    x = x+1

Scoring by matching the number of words that match the MAtch_WORDS column of secondary

In [271]:
x = 0
while x<len(secondary):
    y = 0
    while y<len(secondary.MAtCH_WORDS[x]):
        z =0
        while z<len(question_bank):
            if question_bank.question[z].find(secondary.MAtCH_WORDS[x][y])!= -1:
                question_bank[secondary.TOPIC[x]][z] = question_bank[secondary.TOPIC[x]][z]+1
            z= z+1
        y =y+1
    x = x+1

C:\Users\hp\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


After sorting we can get the highly frequent questions that are asked in interviews 

In [273]:
q = question_bank.copy()

In [284]:
q.sort_values('inheritance',axis=0,ascending=False)

,question,fcfs,dml,jdbc,joints,buffering,applet,deadlock,ddl,polymorphism,caching,thread,security,scheduling,inheritance,encapsulation,transaction,object,sql,keys
490,1. java basics what is object class abstract...,0,0,0,1,0,2,0,0,6,0,2,0,0,6,4,0,2,1,0
118,7. he asked about inheritance polymorphism a...,0,1,1,1,0,0,1,0,5,0,0,1,0,6,5,0,0,1,1
112,correct answer: data abstraction data encapsu...,0,2,0,0,1,0,0,0,4,1,1,1,0,5,5,1,1,1,1
484,7. he asked about inheritance polymorphism a...,0,1,1,1,0,0,1,0,4,0,0,1,0,5,4,0,0,1,1
19,some background on this: c++ supports multiple...,0,1,0,1,0,0,0,0,1,0,0,0,1,5,1,0,1,0,1
349,he: can you explain how would you use java oop...,0,0,2,2,0,1,1,0,6,0,3,0,0,5,5,0,0,2,1
256,correct answer: data abstraction data encapsu...,0,2,0,0,1,0,0,0,4,1,1,1,0,5,5,1,1,1,1
185,3) core java concepts like inheritance polymo...,0,0,1,3,0,1,0,0,4,0,1,0,0,4,2,0,0,2,1
133,tech rnd2: threads again they told to write p...,0,0,1,0,0,0,1,0,3,0,3,0,0,4,3,0,0,0,0
74,a. private: b'coz the private inheritance make...,0,1,1,0,0,0,0,0,1,0,0,0,0,4,1,0,1,0,0
